In [6]:
!pip install  tabulate
!pip install langchain_core langchain_openai langchain_community langgraph langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.0 MB/s eta 0:00:00


In [7]:
import pandas as pd
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.agents.agent_types import AgentType

In [15]:
csv_file_path = '/content/sales_report.csv'

In [9]:
llm = ChatOpenAI(openai_api_base="https://openrouter.ai/api/v1", openai_api_key="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", model="local-model")

/tmp/ipython-input-2393671857.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_base="https://openrouter.ai/api/v1", openai_api_key="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", model="local-model")


In [16]:
print("\nCreating CSV Agent...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # Allow the agent to execute python code
    handle_parsing_errors=True # Handle cases where the LLM output is not perfectly formatted

)
print("Agent is ready.")


Creating CSV Agent...
Agent is ready.


/usr/local/lib/python3.12/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [17]:
AGENT_PREFIX = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you.
You have access to a single tool: `python_repl_ast`.
Use this to process the dataframe and return the final answer.

Your response MUST be in the following format:

Question: the user's question
Thought: I need to use pandas to find the answer. I will write the code to do that.
Action: python_repl_ast
Action Input: a single, valid line of pandas code to run

Example:
Question: how many rows are there?
Thought: I need to count the rows in the dataframe.
Action: python_repl_ast
Action Input: df.shape[0]
"""

print("\nCreating CSV Agent with a guiding prefix...")
agent_executor = create_csv_agent(
    llm,
    csv_file_path,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
    handle_parsing_errors=True,
    prefix=AGENT_PREFIX
)
print("Agent is ready.")


Creating CSV Agent with a guiding prefix...
Agent is ready.


/usr/local/lib/python3.12/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [18]:
query1 = "What was the total sale amount for the North region?"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])



Question: What was the total sale amount for the North region?


> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': 'local-model is not a valid model ID', 'code': 400}, 'user_id': 'user_2zlQXJqakDGCfy9QBqD3HMK79R2'}

In [ ]:
query1 = "how many laptops sold in south region"
print(f"\nQuestion: {query1}")
response1 = agent_executor.invoke(query1)
print("Answer:", response1['output'])

In [19]:
!pip install langchain_experimental